# Libraries import

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing

from pickle import load, dump
from xgboost import XGBRegressor

In [ ]:
os.chdir('\\Users\\iokhotnikov\\Documents\\Python\\hhl\\test_rig\\code')
from scripts.utils.readers import DataReader, Preprocessor
from scripts.utils.config import FEATURES, FEATURES_NO_TIME, MODELS_PATH, PREDICTIONS_PATH

In [ ]:
os.chdir('\\Users\\iokhotnikov\\Documents\\Python\\hhl\\test_rig')

# Data import

In [ ]:
def read_data(mode='preprocessed'):
    if mode == 'raw':
        df = DataReader.read_all_raw_data(verbose=True,
                                          features_to_read=FEATURES)
        df = Preprocessor.remove_step_zero(df, inplace=False)
        df.sort_values(by=['DATE', 'TIME'], inplace=True, ignore_index=True)
    if mode == 'processed':
        df = pd.read_csv(os.path.join('data', 'processed',
                                      'combined_timed_data.csv'),
                         parse_dates=True,
                         infer_datetime_format=True,
                         dtype=dict(
                             zip(FEATURES_NO_TIME,
                                 [np.float32] * len(FEATURES_NO_TIME))))
        df[['STEP', 'UNIT', 'TEST',
            'ARMANI']] = df[['STEP', 'UNIT', 'TEST',
                             'ARMANI']].astype(np.uint8)
        df['TIME'] = pd.to_datetime(df['TIME'])
        df['DATE'] = pd.to_datetime(df['DATE'])
    df['RUNNING TIME'] = pd.date_range(start=f'{min(df["DATE"])} 00:00:00',
                                       periods=len(df),
                                       freq='s')
    df['RUNNING DURATION'] = pd.to_timedelta(range(len(df)), unit='s')
    df['RUNNING HOURS'] = (
        pd.to_timedelta(range(len(df)), unit='s').total_seconds() /
        3600).astype(np.float32)
    return df

In [ ]:
df = read_data(mode='processed')

# Feature engineering

In [ ]:
def plot_feature(df, feature):
    plt.figure(figsize=(20, 5), tight_layout=True)
    plt.plot(df['RUNNING HOURS'], df[feature])
    plt.ylabel(feature)
    plt.xlabel('TIME, HOURS')
    plt.show()


def plot_data(df):
    for feature in df.columns:
        if 'RUNNING' not in feature:
            plot_feature(df, feature)

In [ ]:
# plot_data(df[FEATURES_NO_TIME])

In [ ]:
INITIAL_TREND_FEATURES = [
    'M1 CURRENT', 'M1 TORQUE', 'PT4', 'D1 RPM', 'D1 CURRENT', 'D1 TORQUE',
    'M2 RPM', 'M2 Amp', 'M2 Torque', 'CHARGE PT', 'CHARGE FLOW', 'M3 Amp',
    'M3 Torque', 'Servo PT', 'SERVO FLOW', 'HSU IN', 'TT2', 'HSU OUT',
    'M5 Amp', 'M5 Torque', 'M6 RPM', 'M6 Amp', 'M6 Torque', 'M7 RPM', 'M7 Amp',
    'M7 Torque', 'Vibration 1', ' Vibration 2'
]

In [ ]:
ENGINEERED_FEATURES = [
    'DRIVE POWER', 'LOAD POWER', 'CHARGE MECH POWER', 'CHARGE HYD POWER',
    'SERVO MECH POWER', 'SERVO HYD POWER', 'SCAVENGE POWER',
    'MAIN COOLER POWER', 'GEARBOX COOLER POWER'
]
df['DRIVE POWER'] = (df['M1 SPEED'] * df['M1 TORQUE'] * np.pi / 30 /
                     1e3).astype(np.float32)
df['LOAD POWER'] = abs(df['D1 RPM'] * df['D1 TORQUE'] * np.pi / 30 /
                       1e3).astype(np.float32)
df['CHARGE MECH POWER'] = (df['M2 RPM'] * df['M2 Torque'] * np.pi / 30 /
                           1e3).astype(np.float32)
df['CHARGE HYD POWER'] = (df['CHARGE PT'] * 1e5 * df['CHARGE FLOW'] * 1e-3 /
                          60 / 1e3).astype(np.float32)
df['SERVO MECH POWER'] = (df['M3 RPM'] * df['M3 Torque'] * np.pi / 30 /
                          1e3).astype(np.float32)
df['SERVO HYD POWER'] = (df['Servo PT'] * 1e5 * df['SERVO FLOW'] * 1e-3 / 60 /
                         1e3).astype(np.float32)
df['SCAVENGE POWER'] = (df['M5 RPM'] * df['M5 Torque'] * np.pi / 30 /
                        1e3).astype(np.float32)
df['MAIN COOLER POWER'] = (df['M6 RPM'] * df['M6 Torque'] * np.pi / 30 /
                           1e3).astype(np.float32)
df['GEARBOX COOLER POWER'] = (df['M7 RPM'] * df['M7 Torque'] * np.pi / 30 /
                              1e3).astype(np.float32)

In [ ]:
TREND_FEATURES = ENGINEERED_FEATURES + [
    'PT4', 'HSU IN', 'TT2', 'HSU OUT', 'Vibration 1', ' Vibration 2'
]

In [ ]:
test_lengths = []
for unit in df['UNIT'].unique():
    for unit_test in df[df['UNIT'] == unit]['TEST'].unique():
        test_lengths.append(
            len(df[(df['UNIT'] == unit) & (df['TEST'] == unit_test)]))
mean_test_dur_sec = np.mean(test_lengths)
print(
    f'Mean test duration {mean_test_dur_sec:.2f} seconds = {mean_test_dur_sec/60:.2f} minutes = {mean_test_dur_sec/3600:.2f} hours'
)

In [ ]:
def decompose_plot(df, period, plot=True):
    if plot:
        fig, ax = plt.subplots(nrows=len(TREND_FEATURES),
                               ncols=4,
                               sharex=True,
                               figsize=(30, 60))

    for i, column in enumerate(TREND_FEATURES):
        res = seasonal_decompose(df[column],
                                 period=int(period),
                                 model='additive',
                                 extrapolate_trend='freq')

        df[f'{column} trend'] = res.trend
        df[f'{column} seasonal'] = res.seasonal

        if plot:
            ax[i, 0].set_title(f'Decomposition of {column}', fontsize=16)
            res.observed.plot(ax=ax[i, 0], legend=False)
            ax[i, 0].set_ylabel('Observed', fontsize=14)
            res.trend.plot(ax=ax[i, 1], legend=False)
            ax[i, 1].set_ylabel('Trend', fontsize=14)
            res.seasonal.plot(ax=ax[i, 2], legend=False)
            ax[i, 2].set_ylabel('Seasonal', fontsize=14)
            res.resid.plot(ax=ax[i, 3], legend=False)
            ax[i, 3].set_ylabel('Residual', fontsize=14)
    if plot:
        plt.show()

In [ ]:
decompose_plot(df, mean_test_dur_sec, plot=False)

In [ ]:
FINAL_TREND_FEATURES = TREND_FEATURES + [
    f for f in df.columns if f.endswith('trend')
]

# Trend forecasting with `statsmodels`

In [ ]:
def plot_ma_mean_std(df, window):
    for feature in df.columns:
        if 'RUNNING' not in feature:
            plt.figure(figsize=(20, 5), tight_layout=True)
            plt.plot(df['RUNNING HOURS'],
                     df[feature],
                     label='observed',
                     color='steelblue')
            plt.plot(df['RUNNING HOURS'],
                     df[feature].rolling(window=int(window)).mean(),
                     label='mean moving average',
                     color='indianred')
            plt.plot(df['RUNNING HOURS'],
                     df[feature].rolling(window=int(window)).std(),
                     label='std moving average',
                     color='seagreen')
            plt.ylabel(feature)
            plt.xlabel('RUNNING HOURS')
            plt.legend()
            plt.show()

In [ ]:
# plot_ma_mean_std(df[FINAL_TREND_FEATURES], mean_test_dur_sec)

## Stationarity check with Augmented Dickey-Fuller unit root test

In [ ]:
def plot_adfuller_results(df):
    for feature in df.columns:
        if 'RUNNING' not in feature:
            result = adfuller(df[feature].values)
            significance_level = 0.05
            adf_stat = result[0]
            p_val = result[1]
            crit_val_1 = result[4]['1%']
            crit_val_5 = result[4]['5%']
            crit_val_10 = result[4]['10%']

            if (p_val < significance_level) & (adf_stat < crit_val_1):
                linecolor = 'forestgreen'
            elif (p_val < significance_level) & (adf_stat < crit_val_5):
                linecolor = 'orange'
            elif (p_val < significance_level) & (adf_stat < crit_val_10):
                linecolor = 'red'
            else:
                linecolor = 'purple'
            plt.figure(figsize=(20, 5), tight_layout=True)
            plt.plot(df['RUNNING HOURS'], df[feature], color=linecolor)
            plt.title(
                f'ADF Statistic {adf_stat:0.2f}, p-value: {p_val:0.2f}\nCritical Values 1%: {crit_val_1:0.2f}, 5%: {crit_val_5:0.2f}, 10%: {crit_val_10:0.2f}',
                fontsize=14)
            plt.ylabel(feature, fontsize=14)
            plt.show()

In [ ]:
# plot_adfuller_results(df[FINAL_TREND_FEATURES])

## Exponential smoothing

In [ ]:
def plot_rolling_forecast(df, feature, forecast, ma_window):
    plt.figure(figsize=(20, 5), tight_layout=True)
    plt.plot(df['RUNNING HOURS'], df[feature], label='observed')
    plt.plot(df['RUNNING HOURS'],
             df[feature].rolling(window=int(ma_window)).mean(),
             label='moving average')
    plt.plot(test['RUNNING HOURS'], forecast, label='forecast')
    plt.xlabel('RUNNING HOURS')
    plt.ylabel(feature)
    plt.legend()
    plt.show()

In [ ]:
train = df.iloc[:int(0.8 * len(df))][FINAL_TREND_FEATURES +
                                     ['RUNNING HOURS', 'RUNNING TIME']]
test = df.iloc[int(0.8 * len(df)):][FINAL_TREND_FEATURES +
                                    ['RUNNING HOURS', 'RUNNING TIME']]


In [ ]:
feature = 'Vibration 1 trend'
train.set_index('RUNNING TIME', inplace=True, drop=False)
test.set_index('RUNNING TIME', inplace=True, drop=False)
model = ExponentialSmoothing(train[feature],
                             freq='S',
                             trend='add',
                             damped_trend=True).fit(smoothing_level=.5,
                                                    optimized=True)
forecast = model.forecast(len(test))
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [ ]:
plot_rolling_forecast(df, feature, forecast, mean_test_dur_sec)

In [ ]:
mean_squared_error(forecast, test[feature], squared=False)

In [ ]:
mean_squared_error(
    df[feature].rolling(window=int(mean_test_dur_sec)).mean()[-len(forecast):],
    test[feature],
    squared=False)


# Timeseries forecasting

In [ ]:
train = df.iloc[:int(0.8 * len(df))][FINAL_TREND_FEATURES +
                                     ['RUNNING HOURS', 'RUNNING TIME']]
test = df.iloc[int(0.8 * len(df)):][FINAL_TREND_FEATURES +
                                    ['RUNNING HOURS', 'RUNNING TIME']]
X = train.loc[:, 'RUNNING HOURS'].values
y = train.loc[:, 'Vibration 1'].values
X_test = test.loc[:, 'RUNNING HOURS'].values
y_test = test.loc[:, 'Vibration 1'].values


In [ ]:
def plot_train_val(X_train, X_val, y_train, y_val, feature):
    plt.figure(figsize=(20, 5), tight_layout=True)
    plt.plot(X_train, y_train, color='steelblue', label='train')
    plt.plot(X_val, y_val, color='darkorange', label='val')
    plt.xlim(X[0], X[-1])
    plt.xlabel('RUNNING HOURS')
    plt.ylabel(feature)
    plt.legend()
    plt.show()

In [ ]:
feature = 'Vibration 1'
plot = False

In [ ]:
for idx, (train_idx,
          val_idx) in enumerate(TimeSeriesSplit(n_splits=5).split(X)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    if plot:
        plot_train_val(X_train, X_val, y_train, y_val, feature)
    model = XGBRegressor(
        objective='reg:squarederror',
        n_estimators=1000,
        learning_rate=0.10,
        subsample=0.5,
        colsample_bytree=1,
        max_depth=5,
    )
    model.fit(X_train.reshape(-1, 1),
              y_train.reshape(-1, 1),
              eval_set=[(X_val.reshape(-1, 1), y_val.reshape(-1, 1))],
              eval_metric='rmse',
              verbose=True)
dump(model, open(os.path.join(MODELS_PATH, 'xgb_vibr_1.pkl'), 'wb'))

In [ ]:
forecast = model.predict(X_test.reshape(-1, 1))
plot_rolling_forecast(df, feature, forecast, mean_test_dur_sec)